# TP : Régression Logistique (from scratch) — Méthode de Newton / IRLS

Dans ce notebook, nous implémentons la régression logistique utilisant la méthode de Newton (aussi appelée IRLS : Iteratively Reweighted Least Squares).

Cette méthode repose sur :
- la log-vraisemblance,
- son gradient,
- sa matrice Hessienne,
- et la mise à jour de Newton-Raphson.

Elle converge beaucoup plus rapidement que la descente de gradient.


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression

df = pd.read_csv('diabetes.csv')
feature = 'Glucose'
X = df[[feature]].values.astype(float)
y = df['Outcome'].values.astype(int)
scaler = StandardScaler()
Xs = scaler.fit_transform(X).reshape(-1)
X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.5, random_state=0)


## Modèle et rappels mathématiques

Nous gardons le même modèle :

$$
p_i = \sigma(\beta_0 + \beta_1 x_i).
$$

La log-vraisemblance :

$$
\ell(\beta)
= \sum_{i=1}^n \left[
y_i \log(p_i) + (1-y_i)\log(1-p_i)
\right].
$$

### Gradient
Déjà dérivé :

$$
g(\beta)
=
\begin{pmatrix}
\sum (y_i - p_i) \\
\sum x_i (y_i - p_i)
\end{pmatrix}.
$$

### Hessienne

On dérive à nouveau :

$$
p_i(1-p_i) = w_i \quad\text{(poids IRLS)}.
$$

La Hessienne de $\ell(\beta)$ vaut :

$$
H(\beta)
=
-\sum_{i=1}^n
w_i
\begin{pmatrix}
1 & x_i \\
x_i & x_i^2
\end{pmatrix}.
$$

La mise à jour de Newton pour **maximiser** $\ell$ est :

$$
\beta^{(t+1)} = \beta^{(t)} - H(\beta^{(t)})^{-1} g(\beta^{(t)}).
$$

C’est l’algorithme IRLS classique.


## Interprétation de la méthode IRLS (Iteratively Reweighted Least Squares)

La méthode de Newton appliquée à la régression logistique est aussi appelée **IRLS**, pour *Iteratively Reweighted Least Squares* (Moindres Carrés Pondérés Itératifs).

Cela signifie que, à chaque itération, l’algorithme résout un problème de moindres carrés où chaque observation est pondérée par un poids :

$$
w_i = p_i(1 - p_i).
$$

Ces poids dépendent de la probabilité prédite au point courant, d’où le terme *iteratively reweighted*.

Concrètement, l’algorithme IRLS consiste à résoudre à chaque itération un système linéaire de la forme :

$$
(X^\top W X)\,\Delta = X^\top (y - p),
$$

ce qui correspond exactement à la mise à jour de Newton pour maximiser la log-vraisemblance :

$$
\beta^{(t+1)} = \beta^{(t)} - H(\beta^{(t)})^{-1}\nabla\ell(\beta^{(t)}),
$$

où la Hessienne est donnée par :

$$
H(\beta)
= -\sum_{i=1}^n w_i
\begin{pmatrix}
1 & x_i \\
x_i & x_i^2
\end{pmatrix}.
$$

IRLS est très efficace car il convergé généralement en très peu d’itérations, contrairement à la descente de gradient.


## Implémentation des fonctions nécessaires

Nous définissons :
- la sigmoïde,
- la log-vraisemblance,
- le gradient,
- la Hessienne,
- et l'algorithme IRLS.

In [7]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def log_vraisemblance(beta, X, y):
    # beta: array([b0, b1])
    z = beta[0] + beta[1]*X
    p = sigmoid(z)
    eps = 1e-12
    return np.sum(y*np.log(p+eps) + (1-y)*np.log(1-p+eps))

def hessienne(beta, X):
    z = beta[0] + beta[1]*X
    p = sigmoid(z)
    w = p * (1-p)
    H00 = -np.sum(w)
    H01 = -np.sum(X * w)
    H11 = -np.sum((X**2) * w)
    H = np.array([[H00, H01],
                  [H01, H11]])
    return H

def newton_irls(X, y, max_iter=20, tol=1e-8, damping=0.0, verbose=False):
    beta = np.zeros(2)
    for k in range(max_iter):
        z = beta[0] + beta[1]*X
        p = sigmoid(z)
        g0 = np.sum(y - p)
        g1 = np.sum(X * (y - p))
        g = np.array([g0, g1])
        H = hessienne(beta, X)
        H_reg = H.copy()
        if damping>0:
            H_reg[0,0] -= damping
            H_reg[1,1] -= damping
        try:
            delta = np.linalg.solve(H_reg, g)  # résourdre H delta = g
        except np.linalg.LinAlgError:
            print("Hessian singular, add damping")
            H_reg = H_reg - np.eye(2)*1e-6
            delta = np.linalg.solve(H_reg, g)
        beta_new = beta - delta
        if verbose:
            print(f"iter {k}: beta={beta}, ll={log_vraisemblance(beta,X,y)}")
        if np.linalg.norm(beta_new - beta) < tol:
            beta = beta_new
            break
        beta = beta_new
    return beta

#Entrainement du modéle avec Newton / IRLS
beta_newton = newton_irls(X_train, y_train, max_iter=50, damping=1e-6, verbose=True)
beta_newton


iter 0: beta=[0. 0.], ll=-266.168517334251
iter 1: beta=[-0.59560932  0.85801941], ll=-210.1208406710374
iter 2: beta=[-0.7341563   1.11428207], ll=-207.55866553749044
iter 3: beta=[-0.75085317  1.14640865], ll=-207.52853928549837
iter 4: beta=[-0.75108412  1.14686894], ll=-207.5285334084336
iter 5: beta=[-0.75108417  1.14686904], ll=-207.52853340843336


array([-0.75108417,  1.14686904])

## Comparaison avec une implémentation standard

`sklearn.linear_model.LogisticRegression` utilise en interne une variante de Newton (liblinear ou lbfgs).

Comparer :
- coefficients,
- accuracy,
- AUC,

permet de vérifier la justesse de notre implémentation IRLS from scratch.


In [4]:
# evaluation
def predict(beta, X, thr=0.5):
    return (sigmoid(beta[0] + beta[1]*X) >= thr).astype(int)

print("Newton beta:", beta_newton)
y_pred_test = predict(beta_newton, X_test)
print("Test accuracy (Newton):", accuracy_score(y_test, y_pred_test))
print("Test AUC (Newton):", roc_auc_score(y_test, sigmoid(beta_newton[0] + beta_newton[1]*X_test)))

# comparaison sklearn
clf = LogisticRegression(solver='lbfgs').fit(X_train.reshape(-1,1), y_train)
print("Sklearn beta:", np.array([clf.intercept_[0], clf.coef_[0,0]]))


Newton beta: [-0.75108417  1.14686904]
Test accuracy (Newton): 0.7421875
Test AUC (Newton): 0.7981021633527441
Sklearn beta: [-0.74658524  1.12416839]


## Variance asymptotique des estimateurs
On peut estimer la matrice de covariance asymptotique des estimateurs par l'inverse de l'information de Fisher.

En estimation par Maximum de Vraisemblance :

$$
\mathrm{Var}(\hat{\beta}) \approx I(\hat{\beta})^{-1},
$$

où l'information observée vaut :

$$
I(\beta)
=
\sum_{i=1} w_i
\begin{pmatrix}
1 & x_i \\
x_i & x_i^2
\end{pmatrix},
\qquad
w_i = p_i(1-p_i).
$$

Cela permet d’obtenir :
- erreurs standards,
- intervalles de confiance,
- tests de Wald.


In [14]:
# estimation de la variance asymptotique (information observée)
beta_hat = beta_newton
z = beta_hat[0] + beta_hat[1]*X_train
p = sigmoid(z)
W = p*(1-p)
I_obs = np.array([[np.sum(W), np.sum(X_train*W)],
                  [np.sum(X_train*W), np.sum(X_train**2 * W)]])
cov_beta = np.linalg.inv(I_obs)
se = np.sqrt(np.diag(cov_beta))
print("beta_hat:", beta_hat)
print("SE:", se)
# 95% CI
from scipy.stats import norm
z95 = norm.ppf(0.975)
ci = np.vstack([beta_hat - z95*se, beta_hat + z95*se]).T
print("95% CI:", ci)


beta_hat: [-0.75108417  1.14686904]
SE: [0.12336582 0.14275697]
95% CI: [[-0.99287673 -0.5092916 ]
 [ 0.86707053  1.42666755]]
